In [1]:
#Import der Libraries

%matplotlib inline

import sys
print('Python version:', sys.version)

import IPython
print('IPython:', IPython.__version__)

from time import time

import numpy as np
print('numpy:', np.__version__)

import scipy
print('scipy:', scipy.__version__)

import pandas as pd
print('pandas:', pd.__version__)

import matplotlib.pyplot as plt
#print('matplotlib:', plt.__version__)

import sklearn
print('scikit-learn:', sklearn.__version__)
from sklearn import metrics
from sklearn import ensemble

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.datasets import make_classification
from sklearn.decomposition import PCA

import sklearn.model_selection as ms

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

import seaborn as sns
print('seaborn', sns.__version__)

import nltk

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
#from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

#nltk.download()

import string

Python version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
IPython: 6.1.0
numpy: 1.13.3
scipy: 0.19.1
pandas: 0.20.3
scikit-learn: 0.19.1
seaborn 0.8.0


In [2]:
# Daten einlesen
data_train = pd.read_csv('D:/Projekte/Toxicity NLP/Data/train.csv')
data_test = pd.read_csv('D:/Projekte/Toxicity NLP/Data/test.csv')

# Definiere Features (X) und Labels (y)
X = data_train['comment_text']
y = data_train['toxic']

# Train-Test Split
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, random_state = 1)

### Einzelne Modelle

Nachfolgend werden die einzelnen Modelle mit GridSearch trainiert um geeignete Hyperparamter zu finden. Cross-Validation wird zusätzlich verwendet um die Robustheit der Ergebnisse sicherzustellen. Alle GridSearches werden jedoch unter Beachtung des Berechnungsaufwands und RAM-Bedarfs durchgeführt. Das Hinzufügen eines zusätzlichen Parameters im GridSearch erhöht die Anzahl der Fits um Faktor 2.

In [11]:
# MNB
clf_mnb = Pipeline([('vect', CountVectorizer(token_pattern='[a-zA-Z\'][a-zA-Z\']+', stop_words='english')),
                    ('clf', MultinomialNB())
                    ])
params = {
    'clf__alpha':np.linspace(0.0, 1.0, 10),
}

grid_search = ms.GridSearchCV(clf_mnb, params,n_jobs=-1, verbose=1, cv=3)
t0 = time()
grid_search.fit(X, y)
t1 = time() - t0
print(f"done in {t1:.2f}s")
print()
print(f"Best score: {grid_search.best_score_}")
print(f"Best parameters set: {grid_search.best_params_}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.5min finished


done in 98.25s

Best score: 0.9503355872934305
Best parameters set: {'clf__alpha': 0.66666666666666663}


In [3]:
# Logistische Regression
clf_lr = Pipeline([('vect', CountVectorizer(token_pattern='[a-zA-Z\'][a-zA-Z\']+', stop_words='english')),
                    ('clf', LogisticRegression(random_state=1, n_jobs=-1))
                    ])
params = {
    #'clf__penalty':('l1', 'l2'),
    'clf__C': (0.01, 0.1, 1, 10, 100),
    
}

grid_search = ms.GridSearchCV(clf_lr, params,n_jobs=-1, verbose=1, cv=3)
t0 = time()
grid_search.fit(X, y)
t1 = time() - t0
print(f"done in {t1:.2f}s")
print()
print(f"Best score: {grid_search.best_score_}")
print(f"Best parameters set: {grid_search.best_params_}")

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.4min finished
D:\Software\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


done in 167.37s

Best score: 0.9524788338733229
Best parameters set: {'clf__C': 10}


In [28]:
# Random Forest mit Grid Search
clf_rf = Pipeline([
    ('vect', CountVectorizer(token_pattern='[a-zA-Z\'][a-zA-Z\']+', stop_words='english')),
    ('clf', ensemble.RandomForestClassifier(criterion='entropy',min_samples_leaf=5, random_state=1, max_features=None)),
])

params = {
   'clf__n_estimators': (5, 10),
   'clf__max_depth': (5, 10, 15),
   #'clf__criterion' : ('entropy', 'gini'),
}

grid_search = ms.GridSearchCV(clf_rf, params,n_jobs=-1, verbose=1, cv=2)
t0 = time()
grid_search.fit(X, y)
t1 = time() - t0
print(f"done in {t1:.2f}s")
print()

print(f"Best score: {grid_search.best_score_}")
print(f"Best parameters set: {grid_search.best_params_}")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:  1.4min remaining:   17.3s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.0min finished


done in 215.62s

Best score: 0.9426211529663912
Best parameters set:
{'clf__max_depth': 15, 'clf__n_estimators': 10}


Die Präzision des Random Forest könnte durch Freigabe der limitierenden Parameter zusätzlich erhöht werden. Allerdings steigt die Berechnungsdauer stark an und mein älterer 8GB RAM wird leider gesprengt.  

In [7]:
# RF mit Adaboost
clf_rf = ensemble.RandomForestClassifier(n_estimators=10,max_depth=15,criterion="entropy",max_features=None,random_state=1,n_jobs=-1)

params = {
    'clf_ada__n_estimators' : (1, 2, 5),
}

text_clf6 = Pipeline([('vect', CountVectorizer(token_pattern='[a-zA-Z\'][a-zA-Z\']+', stop_words='english')),
                    ('clf_ada', ensemble.AdaBoostClassifier(clf_rf, n_estimators=5, random_state=1))
                     ])
grid_search = ms.GridSearchCV(text_clf6, params, n_jobs=-1, verbose=1, cv=2)
t0 = time()
grid_search.fit(X, y)
t1 = time() - t0
print(f"done in {t1:.2f}s")
print()
print(f"Best score: {grid_search.best_score_}")
print("Best parameters set:")
print(f"Best parameters set: {grid_search.best_params_}")

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  3.1min finished


done in 334.55s

Best score: 0.9456981531731956
Best parameters set:
Best parameters set: {'clf_ada__n_estimators': 5}


### Ensemble mit VotingClassifier

Für das Ensemble werden die heterogenen Modelle MNB (bessere Performance als BNB), Logistic Regression und Random Forest ausgewählt. Die Modelle sollen sich in ihrer Vorhersagemethodik unterscheiden und möglichst unabhängig voneinander sein. Dadurch werden im Ensemble die spezifischen Schwächen jedes Modells ausgeglichen. Daher wird auf ähnliche Varianten zur Ensemblebildung verzichtet. Dazu zählen z.B. MNB vs BNB und LR vs SVM.

In [10]:
# Modelle mit den besten Parametern aus den GridSearches
clf_mnb = Pipeline([('vect', CountVectorizer(token_pattern='[a-zA-Z\'][a-zA-Z\']+', stop_words='english')),('clf', MultinomialNB(alpha = 0.667))])
clf_lr = Pipeline([('vect', CountVectorizer(token_pattern='[a-zA-Z\'][a-zA-Z\']+', stop_words='english')),('clf', LogisticRegression(C=10))])
clf_rf = ensemble.RandomForestClassifier(n_estimators=10,max_depth=15,min_samples_leaf=5,criterion="entropy",max_features=None,random_state=1,n_jobs=-1)
clf_rf_ada = Pipeline([('vect', CountVectorizer(token_pattern='[a-zA-Z\'][a-zA-Z\']+', stop_words='english')),('clf', ensemble.AdaBoostClassifier(clf_rf, n_estimators=5, random_state=1))])

est = [('mnb', clf_mnb),
        ('lr', clf_lr),
        ('rf_ada', clf_rf_ada)
]

vclf = ensemble.VotingClassifier(estimators=est, n_jobs=-1, voting='hard')

for clf, label in zip([clf_mnb, clf_lr, clf_rf_ada, vclf], ['Multinomial Naive Bayes', 'Logistic Regression', 'Random Forest m. Adaboost', 'Ensemble']):
    scores = ms.cross_val_score(clf, X, y, cv=2, scoring='accuracy')
    print(f"Accuracy: {scores.mean():.4f}: [{label}]")

Accuracy: 0.9498: [Multinomial Naive Bayes]
Accuracy: 0.9529: [Logistic Regression]
Accuracy: 0.9463: [Random Forest m. Adaboost]
Accuracy: 0.9540: [Ensemble]


Das Methodenensemble erzielt damit eine leichte Verbesserung gegenüber den Einzelmethoden. Eine größere Verbesserung könnte durch Verwendung von weiteren Modellen erreicht werden.